In [8]:
'''
Проверяем по id учитель это или ученик
(Реализация данной фугкции подразумевает обязательное наличие учителя на занятии)
'''

def is_tutor(b):  
  global users
  for i in users.iterrows():
    if i[0] > 0 and i[0] < 379:
      str_id = str(users['                  id                  '][i[0]])[1:-1]
      if str_id == b and users[' role  '][i[0]] == ' tutor':
        return True
  return False


'''
Перевод часовой зоны основан на том, что мы уверены, что все наши уроки лежат в одном месяце
'''

def rigth_time(a):
  for i in a.iterrows():
    if i[0] > 0 and i[0] < 379:
      s = a['       scheduled_time       '][i[0]]
      s1 = s[9:11] 
      s2 = s[12:14]
      try:
        s2 = int(s2)
        if s2 >= 3:
          a['       scheduled_time       '][i[0]] = int(s1)
        else:
          a['       scheduled_time       '][i[0]] = int(s1)-1
      except:
        print('Какие-то проблемы с переводом времени')



'''
По event_id находим все участников занятия
'''

def all_participants(event_id):
  global participants
  a = participants[participants[' event_id '] == event_id]
  all = []
  for i in participants.iterrows():
    if i[0]>0 and i[0]< 379 and int(participants[' event_id '][i[0]]) == int(event_id):
      all.append(participants['               user_id                '][i[0]])
  return all


'''
Необходимо по заданному идентификатору урока найти отмети за данный урок
'''

def all_marks_for_lessons(lesson_id):  
  global quality 
  res = []
  a = quality[quality['              lesson_id               '] == lesson_id]
  for i in a.iterrows():
    if i[0] > 0 and i[0] < 379:
      if a[' tech_quality '][i[0]] != None:
        res.append(a[' tech_quality '][i[0]])
  return res



In [9]:
import pandas as pd



# Импортируем библиотеки и читаем файлы
lessons = pd.read_table('lessons.txt', sep = '|')
participants = pd.read_table('participants.txt', sep = '|')
quality = pd.read_table('quality.txt', sep = '|')
users = pd.read_table('users.txt', sep = '|')


d = dict() 
''' В данном словаре в каачестве ключей будут лежать даты, а в качесте значений дикт2,
           в котром ключи- учителя, а значения - массив с их отметками '''


rigth_time(lessons) # приводим к правильному времени
lessons = lessons[lessons[' subject '] == ' phys    '] # отбираем нужные уроки
for i in users.iterrows():
  if i[0] > 0 and i[0] < 379:
    users['                  id                  '][i[0]].replace(' ','')


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [10]:
for i in lessons.iterrows():
  if i[0] > 0 and i[0] < 379:
    id = lessons['                  id                  '][i[0]]
    event_id = str(int(lessons[' event_id '][i[0]]))
    data = lessons['       scheduled_time       '][i[0]]
    all_par = all_participants(event_id)
    for ii in all_par:
      ii = ii[1:]
      if is_tutor(ii):
        tutor = ii
        res_for_less = all_marks_for_lessons(id)
        if data not in d:
          d[data] = dict()
        if tutor not in d[data]:
          d[data][tutor] = []
        for jj in res_for_less:
          try:
            jj.replace(' ','')
            if len(jj) > 0:
              d[data][tutor].append(int(jj))
          except:
            pass




for i in d:
  min = 999
  for j in d[i]:
    res = 0
    n = 0
    for ii in d[i][j]:
      res += int(ii)
      n += 1
    if n != 0:
      d[i][j] = res / n
    else:
      d[i][j] = 999
    if d[i][j] < min:
      min = d[i][j]
      teacher = j
  if min != 999:
    print('2020-01-' + str(i) +  ' ' + str(teacher) +' ' + str(min*10000//100/100) )
  else:
    print('2020-01-' + str(i) + ' ' + str(teacher) + ' 0')


2020-01-19 696c838e-c054-4e9f-a51a-50bf5660f364 4.83
2020-01-17 30a19496-bdaf-461c-abbc-2709ae520201 5.0
2020-01-20 43efce48-94b2-4412-857f-223d45969008 4.5
2020-01-11 8fe03f08-8581-430c-a590-9888ab36deb3 4.42
2020-01-14 c6718d0e-976c-4d6c-b0e0-32c770776567 4.0
2020-01-12 696c838e-c054-4e9f-a51a-50bf5660f364 4.88
2020-01-13 696c838e-c054-4e9f-a51a-50bf5660f364 5.0
2020-01-16 696c838e-c054-4e9f-a51a-50bf5660f364 0
2020-01-15 b37ccae8-fc31-4ad8-8f55-ca855b23fbf6 5.0
2020-01-18 603b8641-c6f6-4d89-ac89-88e50d45aa0d 4.5
